<a href="https://colab.research.google.com/github/kathdevx/emotionalAI/blob/main/Distilorberta_Pretrained_vs_Finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [22]:
import pickle 
import pandas as pd
import glob
import json

In [41]:
test_sentences = []
for file in glob.glob('/content/drive/MyDrive/Data/test sentences with predictions/*'):
    with open(file, 'rb') as infile:
        test_sentences.append(pickle.load(infile))

In [98]:
test_pretrained = {
    'FileName': [],
    'EmoClass':[]
}
for batch,_ in enumerate(test_sentences):
    for input_file,_ in enumerate(test_sentences[batch]):
        test_pretrained['FileName'].append(test_sentences[batch][input_file][0])
        primary_emotion_score = test_sentences[batch][input_file][1][0]['score']
        primary_emotion = test_sentences[batch][input_file][1][0]['label']
        for emotions in test_sentences[batch][input_file][1]:
            if emotions['score'] > primary_emotion_score:
                primary_emotion_score = emotions['score']
                primary_emotion = emotions['label']
                
        test_pretrained['EmoClass'].append(primary_emotion)

In [123]:
import pandas as pd
import glob
path = '/content/drive/MyDrive/Data'


fnames = ['Test1', 'Test2']
dirnames = ['test sentences']

# split_df = None

for i, split in enumerate(fnames):

  split_csv = pd.read_csv(f'{path}/{fnames[i]}.csv')
  split_csv['FileName'] = split_csv['FileName'].str.replace('.wav', '')
  split_csv.drop(split_csv.index[split_csv['EmoClass'] == 'X'], inplace = True)
  split_csv.drop(split_csv.index[split_csv['EmoClass'] == 'O'], inplace = True)
  split_csv.drop(split_csv.index[split_csv['EmoClass'] == 'N'], inplace = True)
  split_csv.drop(split_csv.index[split_csv['EmoClass'] == 'D'], inplace = True)
  split_csv.drop(split_csv.index[split_csv['EmoClass'] == 'C'], inplace = True)

  split_csv = split_csv.reset_index()

  split_df = pd.DataFrame(columns=['FileName', 'EmoClass', 'Sentence'])
  split_df['FileName'] = split_csv['FileName']
  split_df['EmoClass'] = split_csv['EmoClass']
  #   split_df['Path'] = split_csv['Path']

  split_dir = f'{path}/{dirnames[0]}/*'

  total_files = {
    'FileName':[],
    'Sentence': []
  }
  for fname in glob.glob(split_dir):
    with open(fname, 'r') as infile:
      lines = infile.read().splitlines()
                          
      for i,line in enumerate(lines):
        lines[i] = line.replace('.wav', '')
        fname, sentence = lines[i].split(' ',1)
        total_files['FileName'].append(fname)
        total_files['Sentence'].append(sentence)
  sent_df = pd.DataFrame.from_dict(total_files)
                                                                                      
  for i in sent_df.index:
    split_df.loc[split_df['FileName'] == sent_df.loc[i, 'FileName'], 'Sentence'] = sent_df.loc[i, 'Sentence']

    # missing = split_df[split_df['Sentence'].isnull()]
    # missing.to_csv(f'{path}/{split}_missing_sentences.csv', index=False)
  split_df = split_df[~split_df['Sentence'].isnull()]
  split_df = split_df.drop_duplicates(subset=['FileName'])
                                                                                                          
  split_df.to_csv(f'{path}/{split}_sentences_for_pretrained_comparison.csv', index=False)
  print(f'Final {split} Dataset size: {len(split_df)}')
                                                                                                              

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  


Final Test1 Dataset size: 5455
Final Test2 Dataset size: 1523


In [135]:
test1 = pd.read_csv('/content/drive/MyDrive/Data/Test1_sentences_for_pretrained_comparison.csv')
test1 = test1[['FileName', 'EmoClass']]
test2 = pd.read_csv('/content/drive/MyDrive/Data/Test2_sentences_for_pretrained_comparison.csv')
test2 = test2[['FileName', 'EmoClass']]

pretrained_distilrobeta = pd.DataFrame.from_dict(test_pretrained)

In [136]:
pretrained_distilrobeta.drop(pretrained_distilrobeta.index[pretrained_distilrobeta['EmoClass'] == 'love'], inplace = True)
pretrained_distilrobeta = pretrained_distilrobeta.reset_index(drop=True)

In [127]:
pretrained_distilrobeta['EmoClass'].unique()

array(['sadness', 'joy', 'anger', 'fear', 'surprise'], dtype=object)

In [125]:
test1['EmoClass'].unique()

array(['H', 'U', 'F', 'S', 'A'], dtype=object)

In [110]:
emotion_mappings = {
    'anger': 'A',
    'sadness': 'S',
    'joy': 'H',
    'surprise': 'U',
    'fear': 'F'
}

In [137]:
for old_emotion, new_emotion in emotion_mappings.items():
    pretrained_distilrobeta['EmoClass'] = pretrained_distilrobeta['EmoClass'].str.replace(old_emotion, new_emotion)

In [138]:
comparison_df = test1.merge(pretrained_distilrobeta,on='FileName',how='inner').dropna()

In [139]:
comparison_df.head()

,FileName,EmoClass_x,EmoClass_y
0,MSP-PODCAST_0001_0019,H,A
1,MSP-PODCAST_0001_0020,H,A
2,MSP-PODCAST_0001_0025,H,U
3,MSP-PODCAST_0001_0060,H,H
4,MSP-PODCAST_0001_0062,H,H


In [140]:
from sklearn.metrics import classification_report
print(classification_report(comparison_df['EmoClass_x'], comparison_df['EmoClass_y'], target_names=comparison_df['EmoClass_x'].unique(), zero_division=True))

              precision    recall  f1-score   support

           H       0.17      0.64      0.27       915
           U       0.16      0.29      0.20       177
           F       0.75      0.51      0.61      5364
           S       0.23      0.19      0.21       597
           A       0.16      0.02      0.04       913

    accuracy                           0.44      7966
   macro avg       0.29      0.33      0.27      7966
weighted avg       0.57      0.44      0.46      7966



### No common files with Test2 dataset

In [141]:
comparison_df2 = test2.merge(pretrained_distilrobeta,on='FileName',how='inner').dropna()

In [144]:
comparison_df2.head()

,FileName,EmoClass_x,EmoClass_y
